In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu b

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-01-11 00:44:16--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-01-11 00:44:16 (6.60 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VineTable-Analysis-Deliverable2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://databootcamp-mod16dev3.s3.amazonaws.com/vine_table.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("vine_table.csv"), sep=",", header=True, inferSchema=True)
df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2SEAD3ATG0GJA|          5|            2|          2|   N|                Y|
|R3MMKULW7900VH|          3|            1|          1|   N|                Y|
|R169Y8P3SDONXJ|          5|            0|          0|   N|                Y|
|R105X9NK0QVV27|          5|            0|          0|   N|                Y|
|R3N4USRVIGB826|          2|            1|          1|   N|                N|
|R1N402W4Q3SXAG|          5|            2|          5|   N|                Y|
|R18D2LTWPVW6QA|          5|            0|          1|   N|                Y|
| RYSQF2U6LLBR0|          5|            0|          0|   N|                Y|
|R2ARH95GT8GEWA|          5|            2|          2|   N|                Y|
|R3P2HIOQCIN5ZU|          1|            0|          0|   N|     

In [82]:
df3 = df.filter(df["total_votes"] >= 20)
df3.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R701I3ZQH3MYJ|          5|           49|         51|   N|                Y|
|R3F7G2C5PWVTW2|          1|           20|         22|   N|                Y|
|R29UENWYPOAGOL|          3|           48|         50|   N|                Y|
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R12XTRXVY619M6|          5|           25|         27|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
| ROCE1WNET9CYO|          3|           32|         32|   N|                Y|
|  R7N7FZD31XET|          5|           26|         26|   N|                Y|
|R33HHGFPB403GM|          5|           19|         21|   N|     

In [83]:
df4 = df3.filter(df3["helpful_votes"]/df3["total_votes"] >= 0.5)
df4.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R701I3ZQH3MYJ|          5|           49|         51|   N|                Y|
|R3F7G2C5PWVTW2|          1|           20|         22|   N|                Y|
|R29UENWYPOAGOL|          3|           48|         50|   N|                Y|
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R12XTRXVY619M6|          5|           25|         27|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
| ROCE1WNET9CYO|          3|           32|         32|   N|                Y|
|  R7N7FZD31XET|          5|           26|         26|   N|                Y|
|R33HHGFPB403GM|          5|           19|         21|   N|     

In [84]:
vine_y_filter_Y = df4.filter((df4.vine == "Y"))#(df4.verified_purchase == "Y")
vine_y_filter_Y.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2N45ZKRRZS856|          5|           21|         22|   Y|                N|
| R5OMLMK13A8NS|          5|           34|         38|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+



In [85]:
vine_y_filter_N = df4.filter((df4.vine == "N"))
vine_y_filter_N.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R701I3ZQH3MYJ|          5|           49|         51|   N|                Y|
|R3F7G2C5PWVTW2|          1|           20|         22|   N|                Y|
|R29UENWYPOAGOL|          3|           48|         50|   N|                Y|
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R12XTRXVY619M6|          5|           25|         27|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
| ROCE1WNET9CYO|          3|           32|         32|   N|                Y|
|  R7N7FZD31XET|          5|           26|         26|   N|                Y|
|R33HHGFPB403GM|          5|           19|         21|   N|     

In [ ]:
#total number of reviews
from pyspark.sql.functions import count
total_reviews = df.select(count("review_id")).show()

+----------------+
|count(review_id)|
+----------------+
|         1047514|
+----------------+



In [71]:
five_star_df= df4.filter("star_rating == 5")

In [73]:
five_star_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R701I3ZQH3MYJ|          5|           49|         51|   N|                Y|
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R12XTRXVY619M6|          5|           25|         27|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|  R7N7FZD31XET|          5|           26|         26|   N|                Y|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
|R3HNVIIDB6YEGJ|          5|           50|         53|   N|                N|
| R20JASRRHMKRQ|          5|           32|         34|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|     

In [74]:
#the number of 5-star reviews
five_star_df_cnt = five_star_df.select(count("review_id")).show()

+----------------+
|count(review_id)|
+----------------+
|            2001|
+----------------+



In [75]:
five_star_df_paid = five_star_df.filter(five_star_df.vine == "Y")
five_star_df_paid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2N45ZKRRZS856|          5|           21|         22|   Y|                N|
| R5OMLMK13A8NS|          5|           34|         38|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+



In [76]:
five_star_df_paid_cnt=five_star_df_paid.select(count('vine')).show()

+-----------+
|count(vine)|
+-----------+
|          2|
+-----------+



In [88]:
five_star_df_not_paid = five_star_df.filter(five_star_df.vine == "N")
five_star_df_not_paid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R701I3ZQH3MYJ|          5|           49|         51|   N|                Y|
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R12XTRXVY619M6|          5|           25|         27|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|  R7N7FZD31XET|          5|           26|         26|   N|                Y|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
|R3HNVIIDB6YEGJ|          5|           50|         53|   N|                N|
| R20JASRRHMKRQ|          5|           32|         34|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|     

In [90]:
five_star_df_not_paid_cnt=five_star_df_not_paid.select(count('vine')).show()

+-----------+
|count(vine)|
+-----------+
|       1999|
+-----------+



In [91]:
import pyspark.sql.functions as F

five_star_df.groupby('vine').agg(F.count('vine') / five_star_df.count()).show()


+----+--------------------+
|vine|(count(vine) / 2001)|
+----+--------------------+
|   Y|9.995002498750624E-4|
|   N|   0.999000499750125|
+----+--------------------+

